# [EDAMAM API](https://developer.edamam.com/)

### This [nutrition API](https://developer.edamam.com/edamam-docs-nutrition-api) can be used for:  
- Full analysis of food recipes in real time
    - Entity extraction, measure and quantity extraction with computation of the applicable nutrition for the recipe and applicable health and diet labels. 
    - Finally, it adjusts quantity for certain ingredients to account for the cooking process. 
    - For example, it calculates oil absorption for fried recipes, excludes solids from stock and broth recipes, calculates marinate absorption for marinates and much more.
- Extraction of food entities with measures and quantities from unstructured text
- Usage in chatbots transcribing natural speech to text.

This API is paid and requires registration but it offers a free demo with some limitations.

In [1]:
import requests
import json
import pandas as pd

# id and key of the API is stored in a separate file for better security
from credentials import APP_ID, APP_KEY

In [2]:
api_endpoint = 'https://api.edamam.com/api/nutrition-details'

# the docs specify that the request must contain the header : Content-Type: application/json
headers = {
    'Content-type': 'application/json'
}

# we define the recipe as a dictionary
# the key 'title' contains the name of the recipe
# the key 'ingr' is a list of ingredients
recipe = {
    'title': 'Mango milkshake',
    'ingr': ['2 mangoes', '1.5 cups milk', '2 tablespoons sugar']
}

# we submit a POST request to the api endpoint along with:
# parameters containing api_id and api_key
# headers containing the headers for the request
# json containing the recipe dictionary
response = requests.post(api_endpoint,
                         params={'app_id': APP_ID, 'app_key': APP_KEY},
                         headers=headers,
                         json=recipe)

In [3]:
response.status_code

200

In [4]:
data = response.json()
list(data.keys())

['uri',
 'yield',
 'calories',
 'totalWeight',
 'dietLabels',
 'healthLabels',
 'cautions',
 'totalNutrients',
 'totalDaily',
 'ingredients',
 'totalNutrientsKCal']

In [5]:
# uri of the response containing Ontology identifier
data['uri']

'http://www.edamam.com/ontologies/edamam.owl#recipe_7f773c3fe295467bbcccab370ebb4737'

In [6]:
# Number of servings
data['yield']

4.0

In [7]:
# Total energy, kcal
data['calories']

723

In [8]:
# Total weight, g
data['totalWeight']

1062.9999999995773

In [9]:
# Diet labels: “balanced”, “high-protein”, “high-fiber”, “low-fat”, “low-carb”, “low-sodium”
data['dietLabels']  # this recipe does not have any diet label

[]

In [10]:
# Health labels: “vegan”, “vegetarian”, “dairy-free”, “low-sugar”, “low-fat-abs”, “sugar-conscious”,
# “fat free”, “gluten free”, “wheat free”
data['healthLabels']

['VEGETARIAN', 'PEANUT_FREE', 'TREE_NUT_FREE', 'ALCOHOL_FREE']

In [11]:
# Cautions
data['cautions']

['SULFITES']

##### We transpose dataframes that have more columns than rows for better readability

In [12]:
# Total Nutrients
pd.DataFrame(data['totalNutrients']).transpose().head(10)

,label,quantity,unit
ENERC_KCAL,Energy,723.21,kcal
FAT,Fat,14.4486,g
FASAT,Saturated,7.44414,g
FAMS,Monounsaturated,3.91272,g
FAPU,Polyunsaturated,1.19082,g
CHOCDF,Carbs,143.229,g
FIBTG,Fiber,10.752,g
SUGAR,Sugars,135.228,g
SUGAR.added,"Sugars, added",24.95,g
PROCNT,Protein,17.0394,g


In [13]:
# % daily value
pd.DataFrame(data['totalDaily']).transpose().head(10)

,label,quantity,unit
ENERC_KCAL,Energy,36.1605,%
FAT,Fat,22.2286,%
FASAT,Saturated,37.2207,%
CHOCDF,Carbs,47.7429,%
FIBTG,Fiber,43.008,%
PROCNT,Protein,34.0788,%
CHOLE,Cholesterol,12.2,%
NA,Sodium,6.84792,%
CA,Calcium,48.775,%
MG,Magnesium,24.7143,%


##### We had provided natural language input during the request. The API uses Natural Language Processing to identify the details of the ingredients.

In [14]:
# details of mangoes
pd.DataFrame(data['ingredients'][0]['parsed']).transpose()

,0
quantity,2
measure,whole
foodMatch,mangoes
food,mangos
foodId,food_an1dqoib8xj3tyb3pr7c0abi4rbw
weight,672
retainedWeight,672
nutrients,"{'VITD': {'label': 'Vitamin D', 'quantity': 0...."
measureURI,http://www.edamam.com/ontologies/edamam.owl#Me...
status,OK


In [15]:
# details of milk
pd.DataFrame(data['ingredients'][1]['parsed']).transpose()

,0
quantity,1.5
measure,cup
foodMatch,milk
food,whole milk
foodId,food_b49rs1kaw0jktabzkg2vvanvvsis
weight,366
retainedWeight,366
nutrients,"{'VITD': {'label': 'Vitamin D', 'quantity': 18..."
measureURI,http://www.edamam.com/ontologies/edamam.owl#Me...
status,OK


In [16]:
# details of sugar
pd.DataFrame(data['ingredients'][2]['parsed']).transpose()

,0
quantity,2
measure,tablespoon
foodMatch,sugar
food,granulated sugar
foodId,food_axi2ijobrk819yb0adceobnhm1c2
weight,25
retainedWeight,25
nutrients,"{'RIBF': {'label': 'Riboflavin', 'quantity': 0..."
measureURI,http://www.edamam.com/ontologies/edamam.owl#Me...
status,OK


In [17]:
pd.DataFrame(data['totalNutrientsKCal']).transpose()

,label,quantity,unit
ENERC_KCAL,Energy,723,kcal
PROCNT_KCAL,Calories from protein,64,kcal
FAT_KCAL,Calories from fat,122,kcal
CHOCDF_KCAL,Calories from carbohydrates,537,kcal


### Invalid requests

#### Not specifying api_id and api_key

In [18]:
# This is an invalid request as we have not specified api_id and api_key which is required during the request

response = requests.post(api_endpoint,
                         headers=headers,
                         json=recipe)
print(f"Status code: {response.status_code}")
response.json()

Status code: 401


{'error': 'unauthorized'}

#### Not specifying a required parameter ('ingr')

In [19]:
# This is an invalid request as we have not specified any ingredients which is required during the request

response = requests.post(api_endpoint,
                         params={'app_id': APP_ID, 'app_key': APP_KEY},
                         headers=headers,
                         json={'title': 'pizza'})

In [20]:
# we get a status code of 555
# it is not an official http status code but an internal one for this API
# the docs describe it as "Recipe with insufficient quality to process correctly"
print(f"Status code: {response.status_code}")
response.json()

Status code: 555


{'error': 'low_quality'}

In [21]:
# finally let us export the JSON response into a json file for any further use
with open('mango_milkshake_data.json', 'w') as json_file:
    json.dump(data, json_file)